### Use pyhomogenize without any netCDF files; `basics`

Firstly, we want to use pyhomogenize's ``basics`` class. We can create our own time axis and do not need any netCDF files to be read.

In [ ]:
import pyhomogenize as pyh

basics = pyh.basics()

Let's see the default attributes containing a time format for converting strings into ``cftime.datetime`` object, a calendar used for this convertion and a frequncy string for creating a ``CFTimeIndex``. 

In [ ]:
basics.__dict__

We can change the default attributes by calling `basics` again.

In [ ]:
basics = pyh.basics(fmt="%Y%m%d", calendar="noleap")
basics.__dict__

Now, we convert a user-given string into a ``cftime.datetime`` object and vice versa. Here again we can specify the time format and the calendar. Note that the default values will not be overwritten after the conversion.

In [ ]:
date_start = basics.str_to_date("20210101")
date_start

In [ ]:
date_start = basics.str_to_date("2021-01-01", fmt="%Y-%m-%d", calendar="standard")
date_start

In [ ]:
basics.__dict__

By default, ``cftime.datetime`` instance atributes that are not represented by `fmt` will be set to `0` as you can see in the above examples. It is possible to set the unmentioned  instance attributes to the last possible value too. Instance attributes which frequency is higher than a second are still be ignored.

In [ ]:
date_end = basics.str_to_date("20211231", mode="end", calendar="standard")
date_end

Now, let's do it vice versa. Here again, we can specify the string format

In [ ]:
str_start = basics.date_to_str(date_start, fmt="%Y-%m-%d")
str_end = basics.date_to_str(date_end, fmt="%Y-%m-%d")
str_start, str_end

Let's build a ``CFTimeIndex`` with a monthly frequency. As left and right bounds we can use both ``str``'s and ``cftime.datetime`` objects.

In [ ]:
date_range = basics.date_range(start=str_start, end=str_end, frequency="mon")
date_range

In [ ]:
date_range = basics.date_range(start=date_start, end=date_end, frequency="mon")
date_range

Note, if `frequency` equals 'mon' the time steps are set to the middle of the month. To get the beginning or the end of the months use 'MS' os 'M'.

In [ ]:
date_range_start = basics.date_range(start=date_start, end=date_end, frequency="MS")
date_range_start

In [ ]:
date_range_end = basics.date_range(start=date_start, end=date_end, frequency="M")
date_range_end

Now, we can check whether the `date_range`'s values contain the first and the last day of the month.

In [ ]:
basics.is_month_start(date_range)

In [ ]:
basics.is_month_end(date_range)

In [ ]:
basics.is_month_start(date_range_start)

The last ``basics`` method is to crop or limit a date_range to user-specified start and end month values. The new date_range should start with and with the a season.

In [ ]:
date_range_limit = basics.date_range_to_frequency_limits(
    date_range=date_range, smonth=[3, 6, 9, 12], emonth=[2, 5, 8, 11], get_range=True
)
date_range_limit

Instead of specifying a date_range you can specify a start and end date and a frequency

In [ ]:
date_range_limit = basics.date_range_to_frequency_limits(
    start=date_start,
    end=date_end,
    frequency="mon",
    smonth=[3, 6, 9, 12],
    emonth=[2, 5, 8, 11],
    get_range=True,
)
date_range_limit

To get only the left and right borders:

In [ ]:
left, right = basics.date_range_to_frequency_limits(
    start=date_start,
    end=date_end,
    frequency="mon",
    smonth=[3, 6, 9, 12],
    emonth=[2, 5, 8, 11],
)
left, right